# 인터넷에서 지니 탑 200 차트 가져오기

In [1]:
import requests
import pandas as pd

In [3]:
url = 'https://www.genie.co.kr/chart/top200'
req = requests.get(url)
html = req.text
html

'<br>\r\n<br>\r\n<center>\r\n<img src="http://www.geniemusic.co.kr/images/common/logo_r1.png"><br>\r\n<h2> <meta http-equiv="Content-Type" content="text/html;charset=UTF-8"> ì\xa0\x91ì\x86\x8dì\x9a\x94ì²\xadì\x9d´ ë³´ì\x95\x88ì\xa0\x95ì±\x85ì\x97\x90 ì\x9d\x98í\x95´ ì°¨ë\x8b¨ë\x90\x98ì\x97\x88ì\x8aµë\x8b\x88ë\x8b¤. ë\x8b¹ì\x82¬ ê³\xa0ê°\x9dì\x84¼í\x84°ë¡\x9c ë¬¸ì\x9d\x98í\x95´ì£¼ì\x8b\xadì\x8b\x9cì\x98¤.<br><br>\r\nThe security policy of the connection request is blocked. Contact your customer service representative.<br><br>\r\nì§\x80ë\x8b\x88ë®¤ì§\x81 ê³\xa0ê°\x9dì\x84¼í\x84° 1577-5337<br><br>\r\n</h2>\r\n</center>\r\n<br>'

In [28]:
# chrome user agent
header = {'user-agent':
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36"            
}
req = requests.get(url, headers=header)
html=req.text
#html

In [5]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

In [6]:
trs = soup.find_all('tr')
len(trs)

51

In [7]:
tbody = soup.select_one('table.list-wrap').find('tbody')
trs = tbody.select('tr')
len(trs)

50

In [10]:
#크롬 f12 개발자 도구에서 찾아주는 selector
tbody=soup.select_one('#body-content > div.newest-list > div > table > tbody')
trs = tbody.select('tr')
len(trs)

50

- x-path값은 아래와 같으나 BeautifulSoup에서는 지원하지 않음

//*[@id="body-content"]/div[6]/div/table/tbody

### 여러개의 데이터 중 하나를  선택해서 원하는 정보를 추출
- rank, title, artist, album

In [57]:
tr=trs[0]
tr


<tr class="list" songid="94731558">
<td class="check"><input class="select-check" title="나 그댈위해 시 한편을 쓰겠어" type="checkbox"/></td>
<td class="number">151
                                        
                                    <span class="rank">
<span class="rank"><span class="rank-down">13<span class="hide">하강</span></span></span>
</span>
</td>
<td><a class="cover" href="#" onclick="fnViewAlbumLayer('82355601');return false;"><span class="mask"></span><img alt="옛이야기" onerror="this.src='//image.genie.co.kr/imageg/web/common/blank_68.gif';" src="//image.genie.co.kr/Y/IMAGE/IMG_ALBUM/082/355/601/82355601_1635326191417_1_140x140.JPG/dims/resize/Q_80,0"/></a></td>
<td class="link"><a class="btn-basic btn-info" href="#" onclick="fnViewSongInfo('94731558');return false;">곡 제목 정보 페이지</a></td>
<td class="info">
<a class="title ellipsis" href="#" onclick="fnPlaySong('94731558','1');return false;" title="재생">
                                        
                                          

In [18]:
tr.select_one('.number').get_text()

'1\n                                        \n                                    \n유지\n\n'

In [19]:
# rank (split으로 분리해서 첫번째 인덱싱)
a=tr.select_one('.number').get_text().split('\n')
a[0]

'1'

In [20]:
# title 스페이스는 .으로 변경
tr.select_one('.title.ellipsis').get_text()

'\n                                        \n                                            \n                                        \n                                        \n                                        \n                                        \n                                            \n                                                리무진 (Feat. MINO) (Prod. by GRAY)'

In [22]:
# strip으로 띄어쓰기 제거
title = tr.select_one('.title.ellipsis').get_text().strip()
title

'리무진 (Feat. MINO) (Prod. by GRAY)'

In [29]:
# artist
tr.select_one('.artist.ellipsis').get_text().strip()


"BE'O (비오)"

In [30]:
# album
tr.select_one('.albumtitle.ellipsis').get_text().strip()

'쇼미더머니 10 Episode 3'

### 한 페이지에 있는 모든 데이터를 반복문으로 가져와 데이터프레임 만들기

In [31]:
# 개별 리스트로 만들어서 df화
rank_list, title_list, artist_list, album_list = [],[],[],[]
for tr in trs:
    rank=tr.select_one('.number').get_text().split('\n')[0]
    title = tr.select_one('.title.ellipsis').get_text().strip()
    artist = tr.select_one('.artist.ellipsis').get_text().strip()
    album = tr.select_one('.albumtitle.ellipsis').get_text().strip()
    rank_list.append(rank)
    title_list.append(title)
    artist_list.append(artist)
    album_list.append(album)

In [33]:
df = pd.DataFrame({
    '순위':rank_list, '제목':title_list, '가수':artist_list, '앨범':album_list
})
df

,순위,제목,가수,앨범
0,1,리무진 (Feat. MINO) (Prod. by GRAY),BE'O (비오),쇼미더머니 10 Episode 3
1,2,회전목마 (Feat. Zion.T & 원슈타인) (Prod. by Slom),sokodomo,쇼미더머니 10 Episode 2
2,3,만남은 쉽고 이별은 어려워 (Feat. Leellamarz) (Prod. by TOIL),베이식,쇼미더머니 10 Episode 3
3,4,불협화음 (Feat. AKMU) (Prod. by GRAY),Mudd the student,쇼미더머니 10 Semi Final
4,5,쉬어 (Feat. MINO) (Prod. by GRAY),아넌딜라이트 (Anandelight) & Unofficialboyy & BE'O (...,쇼미더머니 10 Episode 1
5,6,사랑은 늘 도망가,임영웅,신사와 아가씨 OST Part.2
6,7,네가 없는 밤 (Feat. ASH ISLAND) (Prod. by GRAY),BE'O (비오),쇼미더머니 10 Final
7,8,strawberry moon,아이유 (IU),strawberry moon
8,9,문득,BE'O (비오),Bipolar
9,10,Wake Up (Prod. by 코드 쿤스트),개코 & Ourealgoat (아우릴고트) & SINCE & 안병웅 & Tabber...,쇼미더머니 10 Episode 1


In [38]:
# 모든 변수 라인으로 받아서 한번에 df화
lines=[]
for tr in trs:
    rank=tr.select_one('.number').get_text().split('\n')[0]
    title = tr.select_one('.title.ellipsis').get_text().strip()
    artist = tr.select_one('.artist.ellipsis').get_text().strip()
    album = tr.select_one('.albumtitle.ellipsis').get_text().strip()
    lines.append([rank,title,artist,album])

df=pd.DataFrame(lines, columns=['순위', '제목','가수','앨범'])
df.head()

,순위,제목,가수,앨범
0,1,리무진 (Feat. MINO) (Prod. by GRAY),BE'O (비오),쇼미더머니 10 Episode 3
1,2,회전목마 (Feat. Zion.T & 원슈타인) (Prod. by Slom),sokodomo,쇼미더머니 10 Episode 2
2,3,만남은 쉽고 이별은 어려워 (Feat. Leellamarz) (Prod. by TOIL),베이식,쇼미더머니 10 Episode 3
3,4,불협화음 (Feat. AKMU) (Prod. by GRAY),Mudd the student,쇼미더머니 10 Semi Final
4,5,쉬어 (Feat. MINO) (Prod. by GRAY),아넌딜라이트 (Anandelight) & Unofficialboyy & BE'O (...,쇼미더머니 10 Episode 1


In [40]:
# 딕셔너리를 만들어서 df화
lines=[]
for tr in trs:
    rank=tr.select_one('.number').get_text().split('\n')[0]
    title = tr.select_one('.title.ellipsis').get_text().strip()
    artist = tr.select_one('.artist.ellipsis').get_text().strip()
    album = tr.select_one('.albumtitle.ellipsis').get_text().strip()
    d = {'순위':rank, '제목': title, '가수':artist, '앨범':album}
    lines.append(d)

df=pd.DataFrame(lines)
df.head()

,순위,제목,가수,앨범
0,1,리무진 (Feat. MINO) (Prod. by GRAY),BE'O (비오),쇼미더머니 10 Episode 3
1,2,회전목마 (Feat. Zion.T & 원슈타인) (Prod. by Slom),sokodomo,쇼미더머니 10 Episode 2
2,3,만남은 쉽고 이별은 어려워 (Feat. Leellamarz) (Prod. by TOIL),베이식,쇼미더머니 10 Episode 3
3,4,불협화음 (Feat. AKMU) (Prod. by GRAY),Mudd the student,쇼미더머니 10 Semi Final
4,5,쉬어 (Feat. MINO) (Prod. by GRAY),아넌딜라이트 (Anandelight) & Unofficialboyy & BE'O (...,쇼미더머니 10 Episode 1


### 모든 페이지의 데이터 가져오기

In [53]:
lines=[]
for page in range(1,5):
    url = f'https://www.genie.co.kr/chart/top200?ditc=D&ymd=20211206&hh=10&rtm=Y&pg={page}'
    req = requests.get(url, headers=header)
    html = req.text
    soup = BeautifulSoup(html, 'html.parser')
    tbody = soup.select_one('table.list-wrap').find('tbody')
    trs = tbody.select('tr')
    
    for tr in trs:
        rank=tr.select_one('.number').get_text().split('\n')[0]
        title = tr.select_one('.title.ellipsis').get_text().strip()
        artist = tr.select_one('.artist.ellipsis').get_text().strip()
        album = tr.select_one('.albumtitle.ellipsis').get_text().strip()
        lines.append([rank,title,artist,album])

In [55]:
df=pd.DataFrame(lines, columns=['순위', '제목','가수','앨범'])
df.head()

,순위,제목,가수,앨범
0,1,리무진 (Feat. MINO) (Prod. by GRAY),BE'O (비오),쇼미더머니 10 Episode 3
1,2,회전목마 (Feat. Zion.T & 원슈타인) (Prod. by Slom),sokodomo,쇼미더머니 10 Episode 2
2,3,만남은 쉽고 이별은 어려워 (Feat. Leellamarz) (Prod. by TOIL),베이식,쇼미더머니 10 Episode 3
3,4,사랑은 늘 도망가,임영웅,신사와 아가씨 OST Part.2
4,5,불협화음 (Feat. AKMU) (Prod. by GRAY),Mudd the student,쇼미더머니 10 Semi Final


In [56]:
df.to_csv('지니차트_top200(211206.csv', index=False)